# 1. CATCHER/SCORE

In [1]:
import psycopg2
import json
import datetime

def get_query(sql, series):
    conn = psycopg2.connect('dbname=baseball host=localhost user=postgres password=postgres')
    cur = conn.cursor()
    cur.execute(sql % (series, series, series, series))
    conn.commit()
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

csvdir = 'C:/Users/ki401/Documents/git/github-io/assets/data/csv'
def replace_table(tablename, filename):
    conn = psycopg2.connect('dbname=baseball host=localhost user=postgres password=postgres')
    cur = conn.cursor()
    trunctb = "TRUNCATE %s;" % tablename
    copytotb = "COPY %s FROM '%s/%s' WITH (format csv, header, encoding 'SJIS');" \
                    % (tablename, csvdir, filename)
    res1 = cur.execute(trunctb)
    res2 = cur.execute(copytotb)
    conn.commit()
    cur.close()
    conn.close()
    return

def get_query_category(sql, series, category):
    conn = psycopg2.connect('dbname=baseball host=localhost user=postgres password=postgres')
    cur = conn.cursor()
    if category=='month':
        category1, category2 = "month::integer", "DATE_PART('month', day_of_game) AS month"
    else:
        category1, category2 = category, category
    cur.execute(sql % (category1, category, category2, series, category, category, category))
    conn.commit()
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results


replace_table('catcher_stats', 'catcher_stats21.csv')
replace_table('stolen_stats', 'stolen_stats21.csv')

sql21v3 = "\
SELECT \
    c.catcher, \
    c.innings, \
    w.win, \
    w.lose, \
    w.save, \
    s.starting, \
    c.runs, \
    c.er, \
    ss.stolen_bases + ss.caught_stealing so, \
    ss.stolen_bases sb, \
    ss.caught_stealing cs \
FROM \
    ( \
        SELECT  \
            catcher, \
            round(sum(floor(innings))+sum(mod(innings,1))*10/3, 2) INNINGS, \
            SUM(runs) RUNS, \
            SUM(earned_runs) ER, \
            ROUND(SUM(earned_runs)/(sum(floor(innings))+sum(mod(innings,1))*10/3)*9*100)/100 ERA \
        FROM catcher_stats \
        WHERE series = '%s' \
        GROUP BY catcher \
    ) c \
    LEFT OUTER JOIN \
        ( \
            SELECT \
                catcher, \
                MAX(CASE tmp.win_lose_save WHEN 'W' THEN tmp.cnt ELSE 0 END) WIN, \
                MAX(CASE tmp.win_lose_save WHEN 'L' THEN tmp.cnt ELSE 0 END) LOSE, \
                MAX(CASE tmp.win_lose_save WHEN 'S' THEN tmp.cnt ELSE 0 END) SAVE \
            FROM (  \
                SELECT catcher, win_lose_save, count(*) cnt FROM catcher_Stats \
                WHERE win_lose_save IS NOT NULL  \
                AND series = '%s'  \
                GROUP BY catcher, win_lose_save  \
            ) tmp \
            GROUP BY catcher \
        ) w \
        ON c.catcher = w.catcher \
    LEFT OUTER JOIN  \
        (SELECT catcher, SUM(stolen_bases) stolen_bases, SUM(caught_stealing) caught_stealing \
         from stolen_stats where series='%s' GROUP BY catcher) ss \
        ON ss.catcher = c.catcher \
    LEFT OUTER JOIN  \
        (SELECT \
            catcher, \
            COUNT(*) starting \
        FROM catcher_stats \
        WHERE starting \
        AND series = '%s'  \
        GROUP BY catcher) s \
        ON s.catcher = c.catcher \
ORDER BY 2 DESC; \
"

sql21_cat = " \
SELECT \
  %s, \
  MAX(CASE tmp2.catcher WHEN '小林' THEN tmp2.wls ELSE NULL END) 小林, \
  MAX(CASE tmp2.catcher WHEN '炭谷' THEN tmp2.wls ELSE NULL END) 炭谷, \
  MAX(CASE tmp2.catcher WHEN '大城' THEN tmp2.wls ELSE NULL END) 大城, \
  MAX(CASE tmp2.catcher WHEN '岸田' THEN tmp2.wls ELSE NULL END) 岸田 \
FROM \
( \
  SELECT \
    catcher, \
    %s, \
    MAX(CASE tmp.win_lose_save WHEN 'W' THEN tmp.cnt ELSE 0 END) || '-' || \
    MAX(CASE tmp.win_lose_save WHEN 'L' THEN tmp.cnt ELSE 0 END) || '-' || \
    MAX(CASE tmp.win_lose_save WHEN 'S' THEN tmp.cnt ELSE 0 END) || ', ' || \
    ROUND(SUM(tmp.er)/sum(tmp.ins)*9, 2) \
     || ' (' || round(sum(tmp.ins)*100)::float/100 || ')' \
    wls \
  FROM ( \
    SELECT  \
      %s, \
      catcher, \
      win_lose_save, \
      COUNT(*) cnt, \
      SUM(earned_runs) er, \
      sum(floor(innings))+sum(mod(innings,1))*10/3 ins \
    FROM catcher_stats \
    WHERE series = '%s' \
    GROUP BY catcher, win_lose_save, %s \
    ORDER BY 1, 2 DESC \
  ) tmp \
  GROUP BY catcher, %s \
  ORDER BY 2, 3 \
) tmp2 \
GROUP BY %s \
ORDER BY 1 ASC, 2 ASC, 3 ASC, 4 ASC  \
;"

def create_table_category(series, category):
    stats = get_query_category(sql21_cat, series, category)

    table_html = ''
    tb = '\t'*11
    for stat in stats:
        table_html += tb+'<tr>\n'
        for x in stat:
            if x is None:
                table_html += tb+'\t<td>---</td>\n'
            else:
                table_html += tb+'\t<td>'+str(x)+'</td>\n'
        table_html += tb+'</tr>\n'
    table_html = table_html[:-1]
    return table_html
    # print(table_html)


stats = get_query(sql21v3, '2021RS')
catcher_stats = []
catcher_stats.append(['捕手', 'イニング', '勝', '敗', 'S', '先発数', '失点', '自責点', '防御率', '盗塁企図', '許盗塁', '盗塁刺', '阻止率'])

for stat in stats:
    catcher_stats.append([x if x else 0 for x in stat])

from decimal import Decimal, ROUND_HALF_UP

for x in catcher_stats[1:]:
    if x[8] > 0:
        x.append(Decimal(x[10]/x[8]).quantize(Decimal('0.001'), rounding=ROUND_HALF_UP))
    else:
        x.append('---')
    x.insert(8, Decimal(x[7]*9/x[1]).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))

table_html = ''
tb = '\t'*11
for catcher_stat in catcher_stats[1:]:
    table_html += tb + '<tr>\n'
    for i, x in enumerate(catcher_stat):
        if x is None:
            table_html += tb + '\t<td></td>\n'
        else:
            if i == 1:
                table_html += tb + '\t<td>'+str(float(x))+'</td>\n'
            else:
                table_html += tb + '\t<td>'+str(x)+'</td>\n'
    table_html += tb + '</tr>\n'
table_html = table_html[:-1]

# pie_data = {'Catcher':['小林', '炭谷', '大城', 'その他'],
pie_data = {'Catcher':['小林', '炭谷', '大城', '岸田'],
            'Stealing_bases':[0,0,0,0],
            'Defense_inning':[
                [0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0]
            ],
            'Starting_games':[
                [0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0]
            ],
            'Winning_games': [
                [0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0]
            ]}

for stat in catcher_stats[1:]:
    pie_data['Stealing_bases'][pie_data['Catcher'].index(stat[0])] = stat[10]
    # if stat[0] in ['小林', '炭谷', '大城']:
    #     pie_data['Stealing_bases'][pie_data['Catcher'].index(stat[0])] = stat[10]
    # else:
    #     pie_data['Stealing_bases'][pie_data['Catcher'].index('その他')] += stat[10]

sql_month_stats = " \
SELECT tmp.month, tmp.catcher, tmp.cnt, tmp.ins, wins.cnt \
from (SELECT  \
    DATE_PART('month', day_of_game)::integer AS month, \
    catcher, \
    sum(case when starting then 1 else 0 end) cnt, \
    round(sum(floor(innings))+sum(mod(innings,1))*10/3, 2)::float ins    \
FROM catcher_stats \
WHERE series = '%s' \
GROUP BY catcher, month \
ORDER BY 1, 2 DESC) tmp \
left outer join ( \
    SELECT  \
      DATE_PART('month', day_of_game) AS month, \
      catcher, \
      COUNT(*) cnt \
    FROM catcher_stats \
    WHERE series = '%s' \
      AND win_lose_save = 'W' \
    GROUP BY catcher, win_lose_save, month \
  ) wins \
  ON tmp.month = wins.month and tmp.catcher=wins.catcher\
;"

def get_query2(sql, series):
    conn = psycopg2.connect('dbname=baseball host=localhost user=postgres password=postgres')
    cur = conn.cursor()
    cur.execute(sql % (series, series))
    conn.commit()
    results = cur.fetchall()
    cur.close()
    conn.close()
    return results

area_datas = get_query2(sql_month_stats, '2021RS')

for x in area_datas:
    pie_data['Starting_games'][pie_data['Catcher'].index(x[1])][x[0]-3] = x[2]
    pie_data['Defense_inning'][pie_data['Catcher'].index(x[1])][x[0]-3] = x[3]
    pie_data['Winning_games'][pie_data['Catcher'].index(x[1])][x[0]-3] = x[4] or 0
    # if x[1] in ['小林', '炭谷', '大城']:
    #     pie_data['Starting_games'][pie_data['Catcher'].index(x[1])][x[0]-3] = x[2]
    #     pie_data['Defense_inning'][pie_data['Catcher'].index(x[1])][x[0]-3] = x[3]
    #     pie_data['Winning_games'][pie_data['Catcher'].index(x[1])][x[0]-3] = x[4] or 0
    # else:
    #     pie_data['Starting_games'][pie_data['Catcher'].index('その他')][x[0]-3] += x[2]
    #     pie_data['Defense_inning'][pie_data['Catcher'].index('その他')][x[0]-3] += x[3]
    #     pie_data['Winning_games'][pie_data['Catcher'].index('その他')][x[0]-3] += x[4] or 0

table_month_html = create_table_category('2021RS', 'month')

table_pitcher_html = create_table_category('2021RS', 'pitcher')

today = datetime.date.today().strftime('%Y.%m.%d')
with open('./index.template.html',mode='r',encoding='utf-8') as f1:
    with open('../../index.html',mode='w',encoding='utf-8') as f2:
        for line in f1:
            f2.write(
                line
                    .replace('@@table_catcher@@', table_html)
                    .replace('@@table_month@@', table_month_html)
                    .replace('@@table_pitcher@@', table_pitcher_html)
                    .replace('@@update_date@@', today)
            )

with open('./chart-pie-demo.template.js',mode='r',encoding='utf-8') as f1:
    with open('../demo/chart-pie-demo.js',mode='w',encoding='utf-8') as f2:
        for line in f1:
            f2.write(
                line.replace('{pie_data21}', str(pie_data))
            )

print(datetime.datetime.now().strftime('%m/%d %H:%M,'),'DATA UPDATE FINISH.')

10/10 17:10, DATA UPDATE FINISH.


In [2]:
# stats = get_query_category(sql21_cat, '2021RS', 'vs_team')
# stats